In [6]:
from keras.models import Sequential  
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2 
import os 
import glob

In [151]:
''' 
    Carregando a pasta de imagens (Dataset dos ovos)
    * A função glob encontra todos os nomes de caminhos que correspondem a um padrão 
      especificado de acordo com as regras usadas pelo shell Unix

    * Para cada um dos arquivos encontrados é feita a leitura
      do arquivo .jpg (imagem), as imagens lidas são armazenadas
      na lista 'data'
'''

img_dir = "C:/Users/marci/Desktop/Projeto mestrado/CNN Egg application/Egg Dataset"
data_path = os.path.join(img_dir,'*g') 

folder = glob.glob(data_path) 
data = [] 
for files in folder: 
    img = cv2.imread(files) 
    data.append(img) 

'''   
    Carregando e tratando o arquivo CSV
'''
eggClass = pd.read_csv("RealData.csv")
cList = eggClass.to_numpy()

print(len(data))
print(cList.shape)

5100
(5100, 1)


In [155]:

sample = np.random.choice(np.arange(5100), 2000, replace = False)
print(sample)

imgList = []
clsList = []

for i in range(2000):

    imgList.append(data[sample[i]])
    clsList.append(cList[sample[i]])

print('class', len(clsList))
print('images', len(imgList))
print('------------------------------------------')
#print('class', clsList)
#print('images', imgList)


[2091 2705 4947 ... 3556 1581  801]
class 2000
images 2000
------------------------------------------


In [162]:
ind = np.arange(len(clsList))
train, test = train_test_split(ind, test_size=0.2, random_state=42)

dataTrain = []
dataTest = []
classTrain = []
classTest = []

for i in range(len(train)):

    dataTrain.append(data[train[i]])
    classTrain.append(cList[train[i]])

for j in range(len(test)):

    dataTest.append(data[test[j]])
    classTest.append(cList[test[j]])

dataTrain = np.array(dataTrain)
dataTest = np.array(dataTest)
classTrain = np.array(classTrain)
classTest = np.array(classTest)

print('img train', len(dataTrain))
print('img test', len(dataTest))
print('class train', len(classTrain))
print('class test', len(classTest))


img train 1600
img test 400
class train 1600
class test 400


In [163]:
'''
    Alterando o tipo dos dados para float32 a fim de aplicar 
    a normalização futuramente
'''

dataTrain = dataTrain.astype('float32')
dataTest = dataTest.astype('float32')

'''
    Realizando a normalização (min/max normalization) a fim de que os valores dos pixels estejam
    entre 0 e 1, tornando o processamento mais eficiente
    obs: 255 é o valor máximo do pixel
'''

dataTrain /= 255
dataTest /= 255


In [4]:
model = Sequential()

# Camada convolucional 1
model.add(Conv2D(96, (11, 11), strides=4, activation='relu', input_shape = (512, 512, 3), padding='valid'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))
    
# Camada convolucional 2
model.add(Conv2D(256, (5, 5), strides=1, activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))
    
# Camada convolucional 3
model.add(Conv2D(384, (3, 3), strides=1, activation='relu', padding='same'))
    
# Camada convolucional
model.add(Conv2D(384, (3, 3), strides=1, activation='relu', padding='same'))
    
# Camada convolucional 5
model.add(Conv2D(256, (3, 3), strides=1, activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=2))
    
# Camada Densa da rede
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units = 1, activation = 'sigmoid'))

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [165]:
'''

   # Definição da rede neural convolucional com arquitetura AlexNet


classifier = Sequential()

# Camada Convolucional 1
classifier.add(Conv2D(96, (11,11), input_shape = (512, 512, 3), activation = 'relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size = (2,2)))

classifier.add(Conv2D(32, (3,3), activation = 'relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size = (2,2)))

classifier.add(Flatten())

    # Adição das hidden Layers

classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dropout(0.2))
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dropout(0.2))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

    # Compilação da rede neural

classifier.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
'''


In [164]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [166]:
''' 
    Etapa de treinamento da rede neural. É importante que a função 'fit_generator' está sendo
    utilizada ao invés da função 'fit' uma vez que ela suporta o processo de augmentation, contudo
    tal função está em processo de depreciação, uma vez que em versões mais atuais, a função 'fit' 
    também suporta.

    Explicação dos parâmetros:

    * trainDatabase -> Dados para treino (após a augmentation)
    * steps_per_epoch -> Número total de etapas (lotes de amostras) a serem produzidas 
                         pelo gerador antes de declarar uma época concluída e iniciar a próxima época.
                         É importante citar que o valor ideal para este parâmetro se dá pela quantidade 
                         total de imagens para treinamento (caso haja um alto poder de processamento) ou
                         pelo total de amostras dividido pel valor do batch_size (caso haja um baixo 
                         poder de processamento)
    * epochs -> Epocas de treinamento da rede
    * validation_data -> Dados para a validação (após a augmentation)
    * validation_steps -> Possui o mesmo princípio do 'steps_per_epoch', porém levando em 
                          consideração a etapa de validação. O valor ideal para este parâmetro 
                          se dá pelo total de amostras dividido pel valor do batch_size
'''
board = TensorBoard(log_dir='./logs')
model.fit(dataTrain, classTrain, batch_size = 16, epochs = 30, validation_data = (dataTest, classTest), callbacks=[board])
#classifier.fit(dataTrain, classTrain, batch_size = 16, epochs = 5, validation_data = (dataTest, classTest))

Epoch 1/5


100/100 [==============================] - 573s 6s/step - loss: 12.7008 - accuracy: 0.6862 - val_loss: 0.6541 - val_accuracy: 0.7500
Epoch 2/5
100/100 [==============================] - 561s 6s/step - loss: 0.5967 - accuracy: 0.7594 - val_loss: 0.5786 - val_accuracy: 0.7500
Epoch 3/5
100/100 [==============================] - 574s 6s/step - loss: 0.5629 - accuracy: 0.7594 - val_loss: 0.5648 - val_accuracy: 0.7500
Epoch 4/5
100/100 [==============================] - 580s 6s/step - loss: 0.5548 - accuracy: 0.7594 - val_loss: 0.5626 - val_accuracy: 0.7500
Epoch 5/5
100/100 [==============================] - 586s 6s/step - loss: 0.5532 - accuracy: 0.7594 - val_loss: 0.5623 - val_accuracy: 0.7500
